<a href="https://colab.research.google.com/github/Arshpreet-Singh-1/ai-machine-learning-for-coders/blob/main/3_3_Horses_vs_Humans_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Imports

In [1]:
import tensorflow as tf
print(tf.__version__)

2.8.2


# 2. Training and Validation Dataset

In [2]:
import urllib.request
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

training_url =  "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"
training_file_name = "horse-or-human.zip"
urllib.request.urlretrieve(training_url, training_file_name)

training_dir  = 'horse-or-human/training/'
zip_ref = zipfile.ZipFile(training_file_name , 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

train_datagen = ImageDataGenerator(
  rescale=1./255.,
  rotation_range=40,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
  fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    training_dir,
    batch_size=20,
    target_size = (150,150),
    class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.


In [3]:
validation_url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip"
validation_file_name = "validation-horse-or-human.zip"
urllib.request.urlretrieve(validation_url , validation_file_name)

validation_dir = 'horse-or-human/validation/'
zip_ref = zipfile.ZipFile(validation_file_name , 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

validation_datagen = ImageDataGenerator(rescale = 1./255.)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    batch_size=20,
    target_size = (150,150),
    class_mode = 'binary'
)

Found 256 images belonging to 2 classes.


# 3. Build and Compile Model

In [4]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras import Model

weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"
urllib.request.urlretrieve(weights_url, weights_file)

pre_trained_model = InceptionV3(input_shape = (150,150,3) , include_top = False , weights = None)
pre_trained_model.load_weights(weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape : ' , last_layer.output_shape)
last_output = last_layer.output

x = layers.Flatten()(last_output)
x = layers.Dense(1024 , activation = 'relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation = 'sigmoid')(x)

model = Model(pre_trained_model.input , x)


last layer output shape :  (None, 7, 7, 768)


In [5]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer = RMSprop(learning_rate = 0.0001) , loss = 'binary_crossentropy' , metrics = ['accuracy'])

# 4. Fit Model

In [6]:
model.fit( train_generator, epochs = 20 , validation_data = validation_generator)

Epoch 1/20
52/52 [==============================] - 19s 259ms/step - loss: 0.1789 - accuracy: 0.9426 - val_loss: 0.0255 - val_accuracy: 0.9883
Epoch 2/20
52/52 [==============================] - 12s 233ms/step - loss: 0.0356 - accuracy: 0.9815 - val_loss: 0.0847 - val_accuracy: 0.9805
Epoch 3/20
52/52 [==============================] - 12s 235ms/step - loss: 0.0131 - accuracy: 0.9951 - val_loss: 2.7415e-04 - val_accuracy: 1.0000
Epoch 4/20
52/52 [==============================] - 13s 248ms/step - loss: 0.0120 - accuracy: 0.9971 - val_loss: 0.0063 - val_accuracy: 0.9961
Epoch 5/20
52/52 [==============================] - 12s 231ms/step - loss: 0.0302 - accuracy: 0.9922 - val_loss: 0.0910 - val_accuracy: 0.9766
Epoch 6/20
52/52 [==============================] - 12s 231ms/step - loss: 0.0120 - accuracy: 0.9951 - val_loss: 0.3791 - val_accuracy: 0.9414
Epoch 7/20
52/52 [==============================] - 12s 233ms/step - loss: 0.0044 - accuracy: 0.9990 - val_loss: 0.0079 - val_accuracy: 0.

# 5. Running the Model

In [7]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
  path = '/content/' + fn
  img = image.load_img(path , target_size = (300,300))
  x = image.img_to_array(img)
  x = np.expand_dims(x , axis = 0)

  image_tensor = np.vstack([x])
  classes = model.predict(image_tensor)
  print(classes)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a horse")
  else:
    print(fn + " is a human")